In [0]:
df = spark.table("classes.gold.fato_consumo_agua")
display(df)

from pyspark.sql import functions as F

df_top10 = (
    df.groupBy("regiao")
      .agg(F.max("valor_total").alias("maior_valor"))
      .orderBy(F.desc("maior_valor"))
      .limit(10)
      .withColumn(
          "maior_valor",
          F.concat(
              F.lit("R$ "),
              F.regexp_replace(
                  F.regexp_replace(F.format_number("maior_valor", 2), ",", "X"),
                  "\\.", ","
              )
          )
      )
      .withColumn(
          "maior_valor",
          F.regexp_replace("maior_valor", "X", ".")
      )
)

display(df_top10)

df_top10.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("classes.gold.top10_maior_valor_FORMATADO")


In [0]:
df = spark.table("classes.gold.fato_consumo_agua")
display(df)

from pyspark.sql import functions as F

df_top10 = (
    df.groupBy("regiao")
      .agg(F.max("valor_total").alias("maior_valor"))
      .orderBy(F.desc("maior_valor"))
      .limit(10)
      .withColumn(
          "maior_valor",
          F.concat(
              F.lit("R$ "),
              F.regexp_replace(
                  F.regexp_replace(F.format_number("maior_valor", 2), ",", "X"),
                  "\\.", ","
              )
          )
      )
      .withColumn(
          "maior_valor",
          F.regexp_replace("maior_valor", "X", ".")
      )
)

display(df_top10)

df_top10.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("classes.gold.top10_maior_valor_FORMATADO")


In [0]:
%sql
SELECT *
FROM classes.gold.top10_regiao_mes_maior_valor
LIMIT 10;


In [0]:
%sql
CREATE OR REPLACE TABLE classes.gold.top10_regiao_mes_maior_valor AS
WITH ranking AS (
    SELECT
        regiao,
        mes_referencia,
        valor_total,
        ROW_NUMBER() OVER (
            PARTITION BY regiao
            ORDER BY valor_total DESC
        ) AS pos
    FROM classes.gold.fato_consumo_agua
),
top1_por_regiao AS (
    SELECT
        regiao,
        mes_referencia,
        valor_total
    FROM ranking
    WHERE pos = 1
)
SELECT
    regiao,
    mes_referencia,
    CONCAT('R$ ', REPLACE(FORMAT_NUMBER(valor_total, 2), '.', ',')) AS valor_total
FROM top1_por_regiao
ORDER BY valor_total DESC
LIMIT 10;b
